In [1]:
import json
import requests

# Ryan code starts

In [ ]:
url = "https://api.covid19api.com/all"
response = requests.get(url)
with open("static/data/covid_all.json", 'w') as jsonfile:
    json.dump(response.json(), jsonfile)

In [ ]:
with open("static/data/covid_all.json") as f:
    covid = json.load(f)

In [ ]:
# code to store data in MongoDB
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
db.historical.drop()

db.historical.insert_many(covid)

In [4]:
confirmed = []
deaths = []
recovered = []

In [ ]:
for i in range(len(covid)):
    data = covid[i]
    if data['Status'] == 'confirmed':
        confirmed.append(data)
    elif data['Status'] == 'deaths':
        deaths.append(data)
    elif data['Status'] == 'recovered':
        recovered.append(data)

In [2]:
def get_date_list(list):
    dates = []
    for i in range(len(list)):
        if list[i]['Date'] not in dates:
            dates.append(list[i]['Date'])
    return sorted(dates)

In [3]:
dates_confirmed = []
dates_deaths = []
dates_recovered = []
dates_confirmed = get_date_list(confirmed)
dates_deaths = get_date_list(deaths)
dates_recovered = get_date_list(recovered)

NameError: name 'confirmed' is not defined

In [ ]:
covid_clean = {
    'confirmed': {},
    'deaths': {},
    'recovered': {}    
}

In [5]:
def get_date_data(dates, data_list, status):
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        covid_clean[status][dates[i][0:10]] = current
    return covid_clean

def get_date_data_nostatus(dates, data_list):
    data_nostatus = dict()
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        data_nostatus[dates[i][0:10]] = current
    return data_nostatus

In [ ]:
covid_clean = get_date_data(dates_confirmed, confirmed, 'confirmed')
covid_clean = get_date_data(dates_deaths, deaths, 'deaths')
covid_clean = get_date_data(dates_recovered, recovered, 'recovered')

In [ ]:
# covid_clean
# for i in range(len(dates_deaths)):
#     print(len(covid_clean['deaths'][dates_deaths[i]]))

In [ ]:
# import pymongo
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize

# db.covid_confirmed.insert_one(covid_clean['confirmed'])
# db.covid_deaths.insert_one(covid_clean['deaths'])
# db.covid_recovered.insert_one(covid_clean['recovered'])

In [6]:
url_confirmed = "https://api.covid19api.com/dayone/country/us/status/confirmed"
url_deaths = "https://api.covid19api.com/dayone/country/us/status/deaths"
url_recovered = "https://api.covid19api.com/dayone/country/us/status/recovered"

In [7]:
response_confirmed = requests.get(url_confirmed).json()
dates_us_confirmed = get_date_list(response_confirmed)
us_confirmed_by_dates = get_date_data_nostatus(dates_us_confirmed, response_confirmed)

with open("static/data/covid_us_confirmed.json", 'w') as jsonfile:
    json.dump(us_confirmed_by_dates, jsonfile)

In [ ]:
date_list = list(us_confirmed_by_dates.keys())
adate = date_list

In [8]:
response_deaths = requests.get(url_deaths).json()
dates_us_deaths = get_date_list(response_deaths)
us_deaths_by_dates = get_date_data_nostatus(dates_us_deaths, response_deaths)

with open("static/data/covid_us_deaths.json", 'w') as jsonfile:
    json.dump(us_deaths_by_dates, jsonfile)

In [9]:
response_recovered = requests.get(url_recovered).json()
dates_us_recovered = get_date_list(response_recovered)
us_recovered_by_dates = get_date_data_nostatus(dates_us_recovered, response_recovered)

with open("static/data/covid_us_recovered.json", 'w') as jsonfile:
    json.dump(us_recovered_by_dates, jsonfile)

In [ ]:
# dates_us_confirmed = get_date_list(covid_confirmed)
# dates_us_deaths = get_date_list(covid_deaths)
# dates_us_recovered = get_date_list(covid_recovered)

In [ ]:
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize
db.us_only_confirmed.drop()
db.us_only_confirmed.insert_one(us_confirmed_by_dates)
db.us_only_deaths.drop()
db.us_only_deaths.insert_one(us_deaths_by_dates)
db.us_only_recovered.drop()
db.us_only_recovered.insert_one(us_recovered_by_dates)

# Ryan code ends

# Kana code starts

In [10]:
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [11]:
# Read unemployment claims data
# Download the updated data here: https://oui.doleta.gov/unemploy/claims.asp
claim_data = pd.read_csv("resources/unemployment_by_state.csv", parse_dates=True)
claim_data.head()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate
0,Alabama,1/4/2020,"4,578",12/28/2019,"18,523","1,923,741",0.96
1,Alabama,1/11/2020,"3,629",1/4/2020,"21,143","1,923,741",1.10
2,Alabama,1/18/2020,"2,483",1/11/2020,"17,402","1,923,741",0.90
3,Alabama,1/25/2020,"2,129",1/18/2020,"18,390","1,923,741",0.96
4,Alabama,2/1/2020,"2,170",1/25/2020,"17,284","1,923,741",0.90


In [12]:
# Select needed columns
claim_data = claim_data[['State','Filed week ended','Initial Claims']]
claim_data.head()

,State,Filed week ended,Initial Claims
0,Alabama,1/4/2020,"4,578"
1,Alabama,1/11/2020,"3,629"
2,Alabama,1/18/2020,"2,483"
3,Alabama,1/25/2020,"2,129"
4,Alabama,2/1/2020,"2,170"


In [13]:
# Replace comma with empty space
claim_data = claim_data.stack().str.replace(',','').unstack()
claim_data.head()

,State,Filed week ended,Initial Claims
0,Alabama,1/4/2020,4578
1,Alabama,1/11/2020,3629
2,Alabama,1/18/2020,2483
3,Alabama,1/25/2020,2129
4,Alabama,2/1/2020,2170


In [15]:
# Change the date format to be able to sort
claim_data['Filed week ended'] = pd.to_datetime(claim_data['Filed week ended'])

# Retrive data only after 1/1/2020
filtered_claim_data = claim_data.loc[claim_data["Filed week ended"] > "2020-01-01"]

# Remove timestamp
filtered_claim_data["Filed week ended"] = filtered_claim_data["Filed week ended"].map(lambda ts: ts.strftime("%Y-%m-%d"))

# Drop na
filtered_claim_data = filtered_claim_data.dropna()
filtered_claim_data.head()

,State,Filed week ended,Initial Claims
0,Alabama,2020-01-04,4578
1,Alabama,2020-01-11,3629
2,Alabama,2020-01-18,2483
3,Alabama,2020-01-25,2129
4,Alabama,2020-02-01,2170


In [ ]:
# Replace virgin island with virgin islands
# filtered_claim_data = filtered_claim_data.replace("Virgin Islands","Virgin Island")

In [ ]:
# Change data type to int
# filtered_claim_data['Initial Claims'] = filtered_claim_data['Initial Claims'].astype(int)

# Check data type again
filtered_claim_data.dtypes

In [16]:
# Create a dictionary for claims by state for each date
df = filtered_claim_data.groupby(["Filed week ended", "State"]).first().reset_index()
datelist = filtered_claim_data["Filed week ended"].unique()
claims_jsonfile = {}

for i in range(len(datelist)):
    claims_jsonfile[datelist[i]] = list()
    for j in range(len(df)):
        if datelist[i] == df["Filed week ended"].iloc[j]:
            state = df['State'][j]
            number = df['Initial Claims'][j]
            claims_jsonfile[datelist[i]].append({state:number}) 
claims_jsonfile

{'2020-01-04': [{'Alabama': '4578'},
  {'Alaska': '981'},
  {'Arizona': '3132'},
  {'Arkansas': '2576'},
  {'California': '36720'},
  {'Colorado': '2431'},
  {'Connecticut': '7852'},
  {'Delaware': '875'},
  {'District of Columbia': '481'},
  {'Florida': '4618'},
  {'Georgia': '17060'},
  {'Hawaii': '1825'},
  {'Idaho': '2957'},
  {'Illinois': '11779'},
  {'Indiana': '4434'},
  {'Iowa': '4586'},
  {'Kansas': '2019'},
  {'Kentucky': '5175'},
  {'Louisiana': '1658'},
  {'Maine': '1663'},
  {'Maryland': '3997'},
  {'Massachusetts': '9464'},
  {'Michigan': '13207'},
  {'Minnesota': '7293'},
  {'Mississippi': '1178'},
  {'Missouri': '7474'},
  {'Montana': '1555'},
  {'Nebraska': '1099'},
  {'Nevada': '2264'},
  {'New Hampshire': '690'},
  {'New Jersey': '15325'},
  {'New Mexico': '799'},
  {'New York': '44846'},
  {'North Carolina': '3508'},
  {'North Dakota': '836'},
  {'Ohio': '10759'},
  {'Oklahoma': '2943'},
  {'Oregon': '6714'},
  {'Pennsylvania': '27298'},
  {'Puerto Rico': '853'},
  

In [ ]:
## Kana to solve mystery later##
# for key, value in states_clean.items():
#     current = []
#     for j in range(len(filtered_claim_data['State'])):
#         if filtered_claim_data['State'].iloc[j] == key:
#             state_claim_dict = {}
#             state_claim_dict[filtered_claim_data['Filed week ended'][j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             states_clean[key] = current
# print(states_clean)

# for key, value in dates_clean.items():
#     current = []
#     for j in range(len(filtered_claim_data['Filed week ended'])):
#         if filtered_claim_data['Filed week ended'].iloc[j] == key:
#             state_claim_dict = {}
#             print(key)
# #             print(filtered_claim_data['Filed week ended'].iloc[j] )
        
# #             print(state_claim_dict[filtered_claim_data['State'].iloc[j]])
#             print(filtered_claim_data['Initial Claims'][j])
#             state_claim_dict[filtered_claim_data['State'].iloc[j]] =filtered_claim_data['Initial Claims'][j]
#             current.append(state_claim_dict)
#             dates_clean[key] = current
# print(dates_clean)

In [17]:
with open('static/data/unemployment_claims.json', 'w') as outfile:
    json.dump(claims_jsonfile, outfile)
# claims_jsonfile.to_json(path_or_buf='unemployment_claims.json')

# Kana code ends

# Connor code starts

# Connor code ends

# Nabeel code starts

# Nabeel code ends